# 🪏 **eXcavation** 🪏

## Package Imports

In [ ]:
# Import Data Management
# ------------------------------------------------------ #
import math
import numpy as np
import pandas as pd
import astropy.units as u
from astropy.io import fits
from astropy.time import Time
from datetime import datetime
from astroquery.irsa import Irsa
# ------------------------------------------------------ #



# Import WCS, Photometry, and Plotting
# ------------------------------------------------------ #
from astropy.wcs import WCS
import matplotlib.pyplot as plt
from astropy.stats import SigmaClip
from matplotlib.patches import Circle
from astropy.coordinates import SkyCoord
from astropy.nddata.utils import Cutout2D
from matplotlib.colors import ListedColormap
from photutils.detection import DAOStarFinder
from photutils.psf import PSFPhotometry, CircularGaussianSigmaPRF
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry, ApertureStats
# ------------------------------------------------------ #



# Import Multithreading and Watching Packages
# ------------------------------------------------------ #
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
# ------------------------------------------------------ #



# Import API Mangament Tools
# ------------------------------------------------------ #
import requests
from io import BytesIO
# ------------------------------------------------------ #



# Plot Style
# ------------------------------------------------------ #
plt.style.use('seaborn-v0_8-paper')
plt.rcParams.update({
      'font.family': 'STIXGeneral',
      'font.size': 14,
      'axes.labelsize': 20,
      'axes.titlesize': 25,
      'xtick.labelsize': 15,
      'ytick.labelsize': 15,
      'axes.linewidth': 1.2,
      'legend.fontsize': 12
  })
  # ------------------------------------------------------ #


## Proper Motion Solution

### Propagate Proper Motion

In [ ]:

# Calculate Decimal Year
#-----------------------------------------------------------------------#
def decimal_year(t):
    year_start = datetime(t.year, 1, 1)
    next_year_start = datetime(t.year + 1, 1, 1)

    year_length = (next_year_start - year_start).total_seconds()
    seconds_into_year = (t - year_start).total_seconds()

    return t.year + seconds_into_year / year_length
#-----------------------------------------------------------------------#



# Calculate Time Passed since CW Observation
#-----------------------------------------------------------------------#
def time(mjd):
    t = Time(mjd, format='mjd')
    decimal_year_cw = t.decimalyear

    now = datetime.now()
    time_passed = decimal_year(now) - decimal_year_cw
    return time_passed
#-----------------------------------------------------------------------#



# Adjust RA and Dec for Proper Motion
#-----------------------------------------------------------------------#
def proper_motion(ra, dec, pmra, pmdec, time_passed):
    dRA  = (pmra  * time_passed) / (3600 * np.cos(np.deg2rad(dec)))
    dDec = (pmdec * time_passed) / 3600

    ra_deg  = ra  + dRA
    dec_deg = dec + dDec
    return ra_deg, dec_deg
#-----------------------------------------------------------------------#


## Photometric Calculations

In [ ]:
def resolving_table(wave):
  if 0.75 <= wave < 2.42:
    R = 41
  elif 2.42 <= wave < 3.82:
    R = 35
  elif 3.82 <= wave < 4.42:
    R = 110
  elif 4.42 <= wave <= 5:
    R = 130
  return wave/R

### Aperature Measurements

In [ ]:

# Perform SphereX Wavelength and Aperature Calculation
# ------------------------------------------------------ #
def spherex_aperature_phot(url, coord, pixel_scale_deg=0.001708333):

    # ----- Download FITS entirely in RAM ----- #
    resp = requests.get(url)
    resp.raise_for_status()
    file_bytes = BytesIO(resp.content)
    # ----------------------------------------- #



    # Opens File
    with fits.open(file_bytes, memmap=False, lazy_load_hdus=True, ignore_missing_simple=True, checksum=False) as hdul:



        # ---------------- WCS ---------------- #
        # Make sure the SIP warning message doesnt show
        spectral_header = hdul["IMAGE"].header.copy()
        for key in list(spectral_header.keys()):
            if key.startswith(("A_", "B_", "AP_", "BP_")):
                del spectral_header[key]

        # Obtain Header Data
        celestial_wcs = WCS(hdul[1].header.copy()) # Normal WCS
        spectral_wcs = WCS(spectral_header, fobj=hdul, key="W") # Wavelength WCS
        flux_image = hdul[1].data.astype(float) - hdul[4].data.astype(float) # IMAGE Image - ZODI Image
        variance_image = hdul['VARIANCE'].data.astype(float) # VARIANCE Image
        # ------------------------------------- #



        # ---------------- Cutouts ---------------- #
        # Cutout of Flux, Variance and WCS Using Inputted Radius
        print(hdul[5].header)
        cutout_flux = Cutout2D(flux_image, position=coord, size=35, wcs=celestial_wcs)
        if cutout_flux.shape[0] != 35 or cutout_flux.shape[1] != 35:
          return {
                    "wavelength": np.nan,
                    "delta_lambda": np.nan,
                    "flux": np.nan,
                    "flux_err": np.nan,
                    "flux_cutout": np.nan,
                    "aperture": np.nan,
                    "annulus": np.nan,
                    "x_loc": np.nan,
                    "y_loc": np.nan,
                    "ap_radius": np.nan,
                    "inner_annulus": np.nan,
                    "outer_annulus": np.nan,
                    "url": url
                }

        cutout_var = Cutout2D(variance_image, position=coord, size=35, wcs=celestial_wcs)
        cutout_wcs = cutout_flux.wcs

        # Convert Flux and Variance Cutouts from MJyr/sr to uJy
        pixel_area_sr = (pixel_scale_deg * np.pi / 180)**2
        flux_ujy = np.nan_to_num(cutout_flux.data * 1e6 * pixel_area_sr * 1e6, nan=0.0)
        variance_ujy = np.nan_to_num(cutout_var.data * (1e6 * pixel_area_sr * 1e6)**2, nan=0.0)
        # ----------------------------------------- #



        # ----- Aperture Photometry With Local Background ----- #
        # Aperature Set-up
        x_query, y_query = cutout_wcs.world_to_pixel(coord) # Puts the RA and DEC into x, y Coord in the Cutout Frame
        fwhm = hdul[1].header['PSF_FWHM']

        r_fwhm = 2.5
        ap_radius_pix = (r_fwhm * fwhm * u.arcsec).to(u.deg).value / pixel_scale_deg # Aperature Radius
        aperture = CircularAperture((x_query, y_query), r=ap_radius_pix) # Set-up Aperature Matrix

        # Annulus Set-up
        an_inner = (45 * u.arcsec).to(u.deg).value / pixel_scale_deg # Inner Radius
        an_outer = (90 * u.arcsec).to(u.deg).value / pixel_scale_deg # Outer Radius
        annulus_ap = CircularAnnulus((x_query, y_query), r_in=an_inner, r_out=an_outer)

        # Calculates Background Frame
        sigclip = SigmaClip(sigma=5.0, maxiters=5) # Calculates the Mean and Standard Deviation of the Background Field
        bkg_per_pix = (ApertureStats(flux_ujy, annulus_ap, sigma_clip=sigclip)).median # Average Annulus Background
        aperture_area = aperture.area_overlap(flux_ujy) # Aperature Area in Pixels
        total_bkg = bkg_per_pix * aperture_area # The Total Background Frame in Aperature Area

        # Calculates the Aperature Photometry
        phot_table = aperture_photometry(flux_ujy, aperture)
        flux_ap = phot_table['aperture_sum'][0] - total_bkg # Aperature - Total Background Frame in Aperature Area

        # Calculate Progated Errors
        phot_var_table = aperture_photometry(variance_ujy, aperture)
        var_flux_ap = phot_var_table['aperture_sum'][0]
        var_bkg_per_pix = (((ApertureStats(flux_ujy, annulus_ap, sigma_clip=sigclip)).std)**2)/(annulus_ap.area_overlap(variance_ujy))
        flux_err = np.sqrt(var_flux_ap + (aperture_area**2)*var_bkg_per_pix)
        # ----------------------------------------------------- #



        # ----- Wavelength ----- #
        # General Set-Up
        y, x = np.indices(hdul["IMAGE"].data.shape) # Obtain Detector Size
        wave, *_ = spectral_wcs.pixel_to_world(x, y) # Convert the x,y Pixels to Wavelength Using Wavelength WCS
        wave = np.asarray(wave)

        # Obtains Wavelength Average
        yslice, xslice = cutout_flux.slices_original
        mask = aperture.to_mask(method='exact')
        wave_cutout = wave[yslice, xslice] # Cutout the Wavelength Cutout Used By Aperature (FIX LATER)
        flux_cutout = flux_image[yslice, xslice] # Cutout the Flux Cutout Used By Aperature (FIX LATER)
        mask = np.isfinite(wave_cutout) & np.isfinite(flux_cutout) & (flux_cutout > 0) # Enough Valid Points
        wave_point = np.average(wave_cutout[mask], weights=flux_cutout[mask]) # Average the Cutouts and Weigh by Flux
        # ---------------------- #

    ap_mask_obj = aperture.to_mask(method='exact')
    an_mask_obj = annulus_ap.to_mask(method='exact')

    # Convert to image-sized arrays
    ap_mask = ap_mask_obj.to_image(cutout_flux.data.shape)
    an_mask = an_mask_obj.to_image(cutout_flux.data.shape)

    # Return Output Dictionary
    return {
        "wavelength": wave_point,
        "delta_lamda": resolving_table(wave_point),
        "flux": flux_ap,
        "flux_err": flux_err,
        "flux_cutout": flux_ujy,
        "aperture": ap_mask,
        "annulus": an_mask,
        "x_loc": x_query,
        "y_loc": y_query,
        "ap_radius": r_fwhm * fwhm,
        "inner_annulus": 45,
        "outer_annulus": 90,
        "url": url
    }
# ------------------------------------------------------ #


### PSF Photometry

In [ ]:

def spherex_psf_phot(url, coord, pixel_scale_deg=0.001708333):
  # ----- Download FITS entirely in RAM ----- #
  resp = requests.get(url)
  resp.raise_for_status()
  file_bytes = BytesIO(resp.content)
  # ----------------------------------------- #



  # Opens File
  with fits.open(url, lazy_load=True) as hdul:



    # ---------------- WCS ---------------- #
    # Make sure the SIP warning message doesnt show
    spectral_header = hdul["IMAGE"].header.copy()
    for key in list(spectral_header.keys()):
        if key.startswith(("A_", "B_", "AP_", "BP_")):
            del spectral_header[key]

    # Obtain Header Data
    celestial_wcs = WCS(hdul[1].header.copy()) # Normal WCS
    spectral_wcs = WCS(spectral_header, fobj=hdul, key="W") # Wavelength WCS
    flux_image = hdul[1].data.astype(float) - hdul[4].data.astype(float) # IMAGE Image - ZODI Image
    variance_image = hdul['VARIANCE'].data.astype(float) # VARIANCE Image
    # ------------------------------------- #



    # ---------------- Cutouts ---------------- #
    # Cutout of Flux, Variance and WCS Using Inputted Radius
    cutout_flux = Cutout2D(flux_image, position=coord, size=20, wcs=celestial_wcs)
    cutout_var = Cutout2D(variance_image, position=coord, size=20, wcs=celestial_wcs)
    cutout_wcs = cutout_flux.wcs

    # Convert Flux and Variance Cutouts from MJyr/sr to uJy
    pixel_area_sr = (pixel_scale_deg * np.pi / 180)**2
    flux_ujy = cutout_flux.data * 1e6 * pixel_area_sr * 1e6
    variance_ujy = cutout_var.data * (1e6 * pixel_area_sr * 1e6)**2
    uncertainty_ujy = np.sqrt(variance_ujy)
    # ----------------------------------------- #



    # ---------------- PSF Photometry ---------------- #
    data = flux_ujy - np.nanmedian(flux_ujy)
    error = uncertainty_ujy

    # PSF parameters
    fwhm = hdul[1].header['PSF_FWHM']/6.15
    sigma = fwhm / 2.355
    bkg_rms = np.nanstd(data)
    finder = DAOStarFinder(5.0 * bkg_rms, fwhm)

    # Pixel-integrated PSF (non-deprecated)
    psf_model = CircularGaussianSigmaPRF(flux=1.0, sigma=sigma)
    psf_model.sigma.fixed = False
    fit_shape = (11, 11)
    psfphot = PSFPhotometry( psf_model, fit_shape, finder=finder, aperture_radius=4)

    phot = psfphot(data, error=error)
    if phot is None:
      flux_point = np.nan
      error_point = np.nan
    elif phot is not None:
      best_idx = np.nanargmin(phot['reduced_chi2'])
      flux_point = phot['flux_fit'][best_idx]
      error_point = phot['flux_err'][best_idx]
    # ------------------------------------------------ #



    # ----- Wavelength ----- #
    # General Set-Up
    y, x = np.indices(hdul["IMAGE"].data.shape) # Obtain Detector Size
    wave, *_ = spectral_wcs.pixel_to_world(x, y) # Convert the x,y Pixels to Wavelength Using Wavelength WCS
    wave = np.asarray(wave)

    # Obtains Wavelength Average
    yslice, xslice = cutout_flux.slices_original
    wave_cutout = wave[yslice, xslice] # Cutout the Wavelength Cutout Used By Aperature (FIX LATER)
    flux_cutout = flux_image[yslice, xslice] # Cutout the Flux Cutout Used By Aperature (FIX LATER)
    mask = np.isfinite(wave_cutout) & np.isfinite(flux_cutout) & (flux_cutout > 0) # Enough Valid Points
    wave_point = np.average(wave_cutout[mask], weights=flux_cutout[mask]) # Average the Cutouts and Weigh by Flux
    # ---------------------- #

    return {
        "wavelength": wave_point,
        "flux": flux_point,
        "flux_err": error_point,
        "flux_cutout": data,
        # "aperture": ap_mask,
        # "annulus": an_mask,
        # "x_loc": x_query,
        # "y_loc": y_query,
        "url": url
    }

## Visual Verification

In [ ]:
def finder_chart(output):
  n_images = len(output['flux_cutout'])
  ncols = 5
  nrows = math.ceil(n_images / ncols)

  wavelengths = np.array(output['wavelength'])
  sorted_idx = np.argsort(wavelengths)
  flux_cutouts_sorted = [output['flux_cutout'][i] for i in sorted_idx]
  apers_sorted = [output['aperture'][i] for i in sorted_idx]
  annuls_sorted = [output['annulus'][i] for i in sorted_idx]
  x_locs_sorted = [output['x_loc'][i] for i in sorted_idx]
  y_locs_sorted = [output['y_loc'][i] for i in sorted_idx]
  wavelengths_sorted = wavelengths[sorted_idx]

  fig, axes = plt.subplots(nrows, ncols, figsize=(3*ncols, 3*nrows), squeeze=False)

  for i in range(n_images):
      row = i // ncols
      col = i % ncols
      ax = axes[row, col]

      cutouts = flux_cutouts_sorted[i]
      apers = apers_sorted[i]
      annuls = annuls_sorted[i]
      x_loc = x_locs_sorted[i]
      y_loc = y_locs_sorted[i]
      wavelength = wavelengths_sorted[i]

      ax.scatter(x_loc, y_loc, c='yellow', marker='*', s = 10)
      ax.imshow(cutouts, cmap='Greys', vmin=np.nanpercentile(cutouts, 1), vmax=np.nanpercentile(cutouts, 99))
      ax.imshow(np.ma.masked_where(apers == 0, apers), cmap=ListedColormap(['red']), alpha=0.2)
      ax.imshow(np.ma.masked_where(annuls == 0, annuls), cmap=ListedColormap(['blue']), alpha=0.2)

      ax.set_title(f"{wavelength:.4f} μm", fontsize=8)
      ax.axis('off')

  for j in range(n_images, nrows*ncols):
      row = j // ncols
      col = j % ncols
      axes[row, col].axis('off')

  plt.tight_layout()
  plt.show()


In [ ]:
def spectra_plot(output):
  plt.figure(figsize=(10,8))

  wavelengths = np.array(output['wavelength'])
  fluxes = np.array(output['flux'])
  errors = np.array(output['flux_err'])

  sort_idx = np.argsort(wavelengths)
  wavelengths = wavelengths[sort_idx]
  fluxes = fluxes[sort_idx]
  errors = errors[sort_idx]

  plt.errorbar(wavelengths, fluxes, yerr=errors, fmt='o', color='slategrey', ecolor='slategrey', elinewidth=1.5, capsize=3, markersize=5)

  plt.plot(wavelengths, fluxes, color='slategrey', alpha=0.6)

  plt.xlabel("Wavelength [μm]", fontsize=14)
  plt.ylabel("Flux [μJy]", fontsize=14)

  plt.grid(alpha=0.2)
  plt.minorticks_on()
  plt.tight_layout()

  plt.ylim(0, 3000)
  plt.show()

## Xcavation

In [ ]:

# Multi-Thread the SphereX Query Function
# ------------------------------------------------------ #
def Xcavation(ra, dec, style, pmra=None, pmdec=None, mjd = None, verification = False, threads = 16):

    # ----- Proper Motion Propagation ----- #
    if pmra is not None and pmdec is not None:
      time_passed = time(mjd) # years since CW observation
      print(f"\nTime Passed Since Observation: {round(time_passed, 6)} years")
      ra_deg, dec_deg = proper_motion(ra, dec, pmra, pmdec, time_passed) # Adjust for Proper Motion
      print(f"Adjusted Coordinates: RA = {round(ra_deg, 6)} deg, Dec = {round(dec_deg, 6)} deg") # Print Adjusted Coordinates
    else:
      ra_deg = ra
      dec_deg = dec
    # ------------------------------------- #



    # ----- SphereX Query ----- #
    coord = SkyCoord(ra_deg, dec_deg, unit='deg') # Convert to astropy Units
    results = Irsa.query_sia(pos=(coord, 10 * u.arcsec), collection='spherex_qr2') # Query SphereX QR2 (Will Need Updates)
    urls = results["access_url"] # Gets the Image URLs
    print(f'\n{len(urls)} Total Images Found in SphereX')
    # ------------------------- #



    # ----- Multi-Thread Wavelength and Aperature Calc. ----- #
    output = []
    with ThreadPoolExecutor(max_workers=threads) as executor:

        print('\nPerforming Photometric Calculations')

        # Multi-Thread w/ tqdm
        if style == 'aperture':
            futures = [executor.submit(spherex_aperature_phot, url, coord) for url in urls]
        elif style == 'psf':
            futures = [executor.submit(spherex_psf_phot, url, coord) for url in urls]
        else:
            return np.nan, np.nan, np.nan

        for future in tqdm(futures):
            result = future.result()

            output.append({
                "wavelength": result["wavelength"],
                "delta_lambda": result["delta_lambda"],
                "flux": result["flux"],
                "flux_err": result["flux_err"],
                "flux_cutout": result["flux_cutout"],
                "aperture": result["aperture"],
                "annulus": result["annulus"],
                "x_loc": result["x_loc"],
                "y_loc": result["y_loc"],
                "ap_radius": result['ap_radius'],
                "inner_annulus": result['inner_annulus'],
                "outer_annulus": result['outer_annulus'],
                "url": result["url"]
            })
    # ------------------------------------------------------- #



    # ----- Finder Chart and Spectral Plots ----- #
    if verification is True:
      finder_chart(output)
      spectra_plot(output)
    # # ------------------------------------------- #



    # Returns The Output Dictionary
    return pd.DataFrame(output)
# ------------------------------------------------------ #


## Example Object

In [ ]:
# MAIN TEST OBJECT
ra = 12.7955492
dec = -15.7382839

style = 'aperture'

# Call function with test input parameters
output = Xcavation(ra, dec, style, threads = 12)

In [ ]:
output['']

In [ ]:
n_images = len(output['flux_cutout'])
ncols = 5
nrows = math.ceil(n_images / ncols)

wavelengths = np.array(output['wavelength'])
sorted_idx = np.argsort(wavelengths)
flux_cutouts_sorted = [output['flux_cutout'][i] for i in sorted_idx]
apers_sorted = [output['aperture'][i] for i in sorted_idx]
annuls_sorted = [output['annulus'][i] for i in sorted_idx]
x_locs_sorted = [output['x_loc'][i] for i in sorted_idx]
y_locs_sorted = [output['y_loc'][i] for i in sorted_idx]
wavelengths_sorted = wavelengths[sorted_idx]
fluxes_sorted = [output['flux'][i] for i in sorted_idx]

aperture_sorted = [output['ap_radius'][i] for i in sorted_idx]
inner_annulus_sorted = [output['inner_annulus'][i] for i in sorted_idx]
outer_annulus_sorted = [output['outer_annulus'][i] for i in sorted_idx]

fig, axes = plt.subplots(nrows, ncols, figsize=(3*ncols, 3*nrows))

axes = axes.ravel()
valid_count = 0
for i in range(n_images):
    if flux_cutouts_sorted[i] is not np.nan:
      ax = axes[i]

      cutouts = flux_cutouts_sorted[i]
      apers = apers_sorted[i]
      annuls = annuls_sorted[i]
      x_loc = x_locs_sorted[i]
      y_loc = y_locs_sorted[i]
      wavelength = wavelengths_sorted[i]
      flux = fluxes_sorted[i]
      aperture = aperture_sorted[i]
      inner_annulus = inner_annulus_sorted[i]
      outer_annulus = outer_annulus_sorted[i]

      ax.scatter(x_loc, y_loc, c='yellow', marker='*', s = 100)
      ax.imshow(cutouts, cmap='Greys', vmin=np.nanpercentile(cutouts, 1), vmax=np.nanpercentile(cutouts, 99))
      ax.imshow(np.ma.masked_where(apers == 0, apers), cmap=ListedColormap(['red']), alpha=0.2)
      ax.imshow(np.ma.masked_where(annuls == 0, annuls), cmap=ListedColormap(['blue']), alpha=0.2)

      ap = Circle((x_loc, y_loc), aperture/6.15, color='red', lw = 2, fill = False)
      in_an = Circle((x_loc, y_loc), inner_annulus/6.15, color='blue', lw = 2, fill = False)
      out_an = Circle((x_loc, y_loc), outer_annulus/6.15, color='blue', lw = 2, fill = False)

      ax.add_patch(ap)
      ax.add_patch(in_an)
      ax.add_patch(out_an)

      ax.set_title(f"$λ$: {wavelength:.4f} μm and $F_λ$: {flux:.0f}", fontsize=12)
      ax.axis('off')
      valid_count += 1

for ax in axes[valid_count:]:
    fig.delaxes(ax)

plt.tight_layout()
plt.subplots_adjust(left=0.02, right=0.98, bottom=0.02, top=0.95, wspace=0.05, hspace=0.15)

pdf_name = "SpUn_320_QA.pdf"
plt.savefig(pdf_name)

files.download(pdf_name)

plt.show()


In [ ]:
plt.figure(figsize=(10,8))

wavelengths = np.array(output['wavelength'])
fluxes = np.array(output['flux'])
errors = np.array(output['flux_err'])

sort_idx = np.argsort(wavelengths)
wavelengths = wavelengths[sort_idx]
fluxes = fluxes[sort_idx]
errors = errors[sort_idx]

plt.errorbar(wavelengths, fluxes, yerr=errors, fmt='o', color='slategrey', ecolor='slategrey', elinewidth=1.5, capsize=3, markersize=5)

plt.plot(wavelengths, fluxes, color='slategrey', alpha=0.6)

plt.xlabel("Wavelength [μm]", fontsize=14)
plt.ylabel("Flux [μJy]", fontsize=14)

plt.grid(alpha=0.2)
plt.minorticks_on()
plt.tight_layout()

plt.ylim(np.nanpercentile(fluxes, 0.01), np.nanpercentile(fluxes, 99.99))
plt.show()

In [ ]:
# import pandas as pd

# df = pd.DataFrame({
#     "wavelength_um": wavelengths,
#     "flux_uJy": fluxes,
#     "flux_err_uJy": errors
# })

# csv_name = "SpUn_320_Aperture.csv"
# df.to_csv(csv_name, index=False)

# files.download(csv_name)

# 🏴‍☠️ Finé